# API import from DST

**The structure of the data gathering and manipulation in this notebook is:**

a. Import packages and initialize the class with the target tables.

b. For each table the steps underneath are present:
1. Table summary
2. Display of variable levels to help you to find the relevant variable codes 
3. Fetch the unit for the table
4. Gathering relevant variables to df                                               *(Obs.: you should change the variable codes in this step if you want different variables)*
5. Changing the df from a long df to a wide one                                     *(Obs.: you can change the variable category in this step if you want different variables)*
6. Export to individual excel file                                                  *(Obs.: you should change the output path in this step)*

c. Combining the data in one excel file.                                            *(Obs.: you should change the output path in this step)*

**a. Import packages and functions from py-file (helper class), using the API**

In [31]:
import requests
import pandas as pd
from IPython.display import display
from io import StringIO
import warnings

In [32]:
# Import the DataPipeline class from data_pipeline.py
from DST_API import DstApi

# Initialize the class with the target tables
MAKRO = DstApi('NAN1')                          #Forsyningsbalance, bruttonationalprodukt (BNP),økonomisk vækst, beskæftigelse mv. efter transaktion og prisenhed
FOU07 = DstApi('FOUOFF07')                      #FoU-omkostninger i den offentlige sektor efter fag og omkostningstyper
FOU09 = DstApi('FOUOFF09')                      #Eksternt finansierede FoU-omkostninger efter sektor, fag og finansieringskilde
Udgifter_FOU = DstApi('RDCE05')                 #Udgifter til FoU efter sektor og videnskabeligt hovedområde

**b. NAN1: Forsyningsbalance, bruttonationalprodukt (BNP),økonomisk vækst, beskæftigelse mv. efter transaktion og prisenhed**

In [33]:
# 1. Get the table summary from the HELPER CLASS
MAKRO.tablesummary(language='en')

Table NAN1: Demand and supply by transaction, price unit and time
Last update: 2024-06-28T08:00:00


,variable name,# values,First value,First value label,Last value,Last value label,Time variable
0,TRANSAKT,31,B1GQK,B.1*g Gross domestic product,EMPM_DC,"Total employment (1,000 persons)",False
1,PRISENHED,6,V_M,"Current prices, (bill. DKK.)",LAN_C,"Pr. capita, 2020-prices, chained values, (1000...",False
2,Tid,58,1966,1966,2023,2023,True


In [34]:
# 2. Get the variable levels from the HELPER CLASS - Use this to figure out whch variables we want to use/get from DST.
MAKRO.variable_levels('TRANSAKT', language='en')
MAKRO.variable_levels('PRISENHED', language='en')
#MAKRO.variable_levels('Tid', language='en')

,id,text
0,V_M,"Current prices, (bill. DKK.)"
1,LAN_M,"2020-prices, chained values, (bill. DKK.)"
2,L_V,Period-to-period real growth (per cent)
3,V_C,"Pr. capita. Current prices, (1000 DKK.)"
4,L_VB,"Contribution to GDP growth, (percentage point)"
5,LAN_C,"Pr. capita, 2020-prices, chained values, (1000..."


In [35]:
# 3. Fetch the unit for the table
unit = MAKRO.get_table_unit(language='en')
print(f"Unit of measurement: {unit}")

Unit of measurement: -


In [36]:
# 4. Define parameters to display and load in from DST
params = {
    'table': 'NAN1',
    'format': 'CSV',
    'variables': [
        {'code': 'TRANSAKT', 'values': ['B1GQK']},
        {'code': 'PRISENHED', 'values': ['V_M', 'LAN_M', 'L_V']},
        {'code': 'Tid', 'values': ['*']}
    ]
}

df_MAKRO = MAKRO.get_data(params=params, as_DataFrame=True)

# Displaying results
print("DataFrame with specified parameters:")

display(df_MAKRO)
# display(df.head(10)) 

DataFrame with specified parameters:


,TRANSAKT,PRISENHED,TID,INDHOLD
0,"B.1*g Bruttonationalprodukt, BNP","Løbende priser, (mia. kr.)",1966,"82,4"
1,"B.1*g Bruttonationalprodukt, BNP","Løbende priser, (mia. kr.)",1967,"90,8"
2,"B.1*g Bruttonationalprodukt, BNP","Løbende priser, (mia. kr.)",1968,"101,3"
3,"B.1*g Bruttonationalprodukt, BNP","Løbende priser, (mia. kr.)",1969,"115,6"
4,"B.1*g Bruttonationalprodukt, BNP","Løbende priser, (mia. kr.)",1970,"128,1"
...,...,...,...,...
169,"B.1*g Bruttonationalprodukt, BNP",Realvækst i forhold til foregående periode (pct.),2019,"1,7"
170,"B.1*g Bruttonationalprodukt, BNP",Realvækst i forhold til foregående periode (pct.),2020,"-1,8"
171,"B.1*g Bruttonationalprodukt, BNP",Realvækst i forhold til foregående periode (pct.),2021,"7,4"
172,"B.1*g Bruttonationalprodukt, BNP",Realvækst i forhold til foregående periode (pct.),2022,"1,5"


In [37]:
# 3. Data manipulation and variable construction

# Pivot the DataFrame
df_MAKRO = df_MAKRO.pivot(index=['TRANSAKT', 'PRISENHED'], columns='TID', values='INDHOLD')

# Reset index to make TRANSAKT and PRISENHED columns again
df_MAKRO = df_MAKRO.reset_index()

# Add the unit information as a new column
if unit:
    df_MAKRO['Enhed'] = unit

# Display the DataFrame
display(df_MAKRO)


TID,TRANSAKT,PRISENHED,1966,1967,1968,1969,1970,1971,1972,1973,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,Enhed
0,"B.1*g Bruttonationalprodukt, BNP","2020-priser, kædede værdier, (mia. kr.)","760,0","804,7","851,8","909,7","926,6","956,0","992,3","1039,2",...,"2152,4","2218,6","2286,4","2328,9","2368,8","2326,6","2498,3","2536,8","2600,1",-
1,"B.1*g Bruttonationalprodukt, BNP","Løbende priser, (mia. kr.)","82,4","90,8","101,3","115,6","128,1","141,7","161,4","185,8",...,"2030,2","2101,5","2189,6","2243,5","2303,6","2326,6","2567,5","2844,2","2804,7",-
2,"B.1*g Bruttonationalprodukt, BNP",Realvækst i forhold til foregående periode (pct.),..,"5,9","5,9","6,8","1,9","3,2","3,8","4,7",...,"2,1","3,1","3,1","1,9","1,7","-1,8","7,4","1,5","2,5",-


In [8]:
# 6. Download data to excel
output_path = "/Users/ssf/Documents/Analyse/Offentlige udgifter/Test export DST/df_MAKRO.xlsx"
df_MAKRO.to_excel(output_path, index=False)

print("DataFrame has been exported to '/Users/ssf/Documents/Analyse/Offentlige udgifter/Test export DST/df_MAKRO.xlsx'.")

DataFrame has been exported to '/Users/ssf/Documents/Analyse/Offentlige udgifter/Test export DST/df_MAKRO.xlsx'.


**b. FOUOFF07: FoU-omkostninger i den offentlige sektor efter fag og omkostningstyper**



In [9]:
# 1. Get the table summary from the HELPER CLASS
FOU07.tablesummary(language='en')

Table FOUOFF07: R&D-expenses in public sector by subject, type of expense and time
Last update: 2023-12-14T08:00:00


,variable name,# values,First value,First value label,Last value,Last value label,Time variable
0,FAGET,64,000,All fields of science,950,Other humanities,False
1,DRIFT1,4,000,R&D full-time employees total,220,Total costs Mio. Dkk,False
2,Tid,16,2007,2007,2022,2022,True


In [10]:
# 2. Get the variable levels from the HELPER CLASS - Use this to figure out whch variables we want to use/get from DST.
FOU07.variable_levels('FAGET', language='en')
FOU07.variable_levels('DRIFT1', language='en')
#FOU07.variable_levels('Tid', language='en')

,id,text
0,000,R&D full-time employees total
1,200,Running costs Mio. Dkk
2,210,Other costs Mio. Dkk
3,220,Total costs Mio. Dkk


In [11]:
# 3. Fetch the unit for the table 
unit = FOU07.get_table_unit(language='en')
print(f"Unit of measurement: {unit}")

Unit of measurement: -


In [12]:
# 4. Define parameters to display and load in from DST
params = {
    'table': 'FOUOFF07',
    'format': 'CSV',
    'variables': [
        {'code': 'FAGET', 'values': ['000']},
        {'code': 'DRIFT1', 'values': ['220']},
        {'code': 'Tid', 'values': ['*']}
    ]
}

df_FOU07 = FOU07.get_data(params=params, as_DataFrame=True)

# Displaying results
print("DataFrame with specified parameters:")

#display(df_FOU07)
display(df_FOU07.head(10)) 

DataFrame with specified parameters:


,FAGET,DRIFT1,TID,INDHOLD
0,Alle fag i alt,Omkostninger i alt Mio. kr.,2007,12788
1,Alle fag i alt,Omkostninger i alt Mio. kr.,2008,14707
2,Alle fag i alt,Omkostninger i alt Mio. kr.,2009,15897
3,Alle fag i alt,Omkostninger i alt Mio. kr.,2010,17413
4,Alle fag i alt,Omkostninger i alt Mio. kr.,2011,18097
5,Alle fag i alt,Omkostninger i alt Mio. kr.,2012,19450
6,Alle fag i alt,Omkostninger i alt Mio. kr.,2013,21013
7,Alle fag i alt,Omkostninger i alt Mio. kr.,2014,20917
8,Alle fag i alt,Omkostninger i alt Mio. kr.,2015,22723
9,Alle fag i alt,Omkostninger i alt Mio. kr.,2016,22773


In [13]:
# 5. Data manipulation and variable construction

# Pivot the DataFrame
df_FOU07 = df_FOU07.pivot(index=['FAGET', 'DRIFT1'], columns='TID', values='INDHOLD')

# Reset index to make TRANSAKT and PRISENHED columns again
df_FOU07 = df_FOU07.reset_index()

# Add the unit information as a new column
if unit:
    df_FOU07['Enhed'] = unit

# Display the DataFrame
display(df_FOU07)

TID,FAGET,DRIFT1,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,Unit
0,Alle fag i alt,Omkostninger i alt Mio. kr.,12788,14707,15897,17413,18097,19450,21013,20917,22723,22773,23524,24647,25374,26044,28740,31507,-


In [14]:
# 6. Download data to excel
output_path = "/Users/ssf/Documents/Analyse/Offentlige udgifter/Test export DST/df_FOU07.xlsx"
df_FOU07.to_excel(output_path, index=False)

print("DataFrame has been exported to '/Users/ssf/Documents/Analyse/Offentlige udgifter/Test export DST/df_FOU07.xlsx'.")

DataFrame has been exported to '/Users/ssf/Documents/Analyse/Offentlige udgifter/Test export DST/df_FOU07.xlsx'.


**b. FOUOFF09: Eksternt finansierede FoU-omkostninger efter sektor, fag og finansieringskilde**

In [15]:
# 1. Get the table summary from the HELPER CLASS
FOU09.tablesummary(language='en')

Table FOUOFF09: External funding of R&D by sector, field of science, source of funding and time
Last update: 2023-12-14T08:00:00


,variable name,# values,First value,First value label,Last value,Last value label,Time variable
0,SEKTOR,5,1000,Sectors total,1400,Private non-profit sector,False
1,VIDOMR,6,10,Natural sciences,60,Humanities,False
2,FINANSKILDE,9,8000,Sources of funding total,8800,Other foreign funding,False
3,Tid,16,2007,2007,2022,2022,True


In [16]:
# 2. Get the variable levels from the HELPER CLASS - Use this to figure out whch variables we want to use/get from DST.
FOU09.variable_levels('SEKTOR', language='en')
FOU09.variable_levels('VIDOMR', language='en')
#FOU09.variable_levels('FINANSKILDE', language='en')
#FOU09.variable_levels('Tid', language='en')

,id,text
0,10,Natural sciences
1,20,Engineering and technology
2,30,Medical sciences
3,40,Agricultural sciences
4,50,Social sciences
5,60,Humanities


In [17]:
# 3. Fetch the unit for the table 
unit = FOU09.get_table_unit(language='en')
print(f"Unit of measurement: {unit}")

Unit of measurement: m DKK


In [18]:
# 4.a Define parameters to display and load in from DST
params = {
    'table': 'FOUOFF09',
    'format': 'CSV',
    'variables': [
        {'code': 'SEKTOR', 'values': ['1000']},
        #{'code': 'VIDOMR', 'values': ['*']},
        {'code': 'FINANSKILDE', 'values': ['*']},
        {'code': 'Tid', 'values': ['*']}
    ]
}

df_FOU09a = FOU09.get_data(params=params, as_DataFrame=True)

# Displaying results
print("DataFrame with specified parameters:")

display(df_FOU09a)
# display(df.head(10)) 

DataFrame with specified parameters:


,SEKTOR,FINANSKILDE,TID,INDHOLD
0,Sektorer i alt,Finansieringskilder i alt,2007,5291
1,Sektorer i alt,Finansieringskilder i alt,2008,5897
2,Sektorer i alt,Finansieringskilder i alt,2009,6811
3,Sektorer i alt,Finansieringskilder i alt,2010,7691
4,Sektorer i alt,Finansieringskilder i alt,2011,7929
...,...,...,...,...
139,Sektorer i alt,Andre udenlandske kilder,2018,618
140,Sektorer i alt,Andre udenlandske kilder,2019,642
141,Sektorer i alt,Andre udenlandske kilder,2020,701
142,Sektorer i alt,Andre udenlandske kilder,2021,570


In [19]:
# 5.a Data manipulation and variable construction

# Pivot the DataFrame
df_FOU09a = df_FOU09a.pivot(index=['SEKTOR', 'FINANSKILDE'], columns='TID', values='INDHOLD')

# Reset index to make TRANSAKT and PRISENHED columns again
df_FOU09a = df_FOU09a.reset_index()

# Add the unit information as a new column
if unit:
    df_FOU09a['Enhed'] = unit

# Display the DataFrame
display(df_FOU09a)

TID,SEKTOR,FINANSKILDE,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,Unit
0,Sektorer i alt,Andre offentlige midler,326,367,515,488,508,555,589,635,676,716,722,684,770,817,691,783,m DKK
1,Sektorer i alt,Andre statslige midler,1273,1587,1643,1749,1867,1915,1953,1940,1918,1732,1774,1654,1521,1515,1599,1536,m DKK
2,Sektorer i alt,Andre udenlandske kilder,255,290,281,369,439,435,476,548,443,412,569,618,642,701,570,612,m DKK
3,Sektorer i alt,Danske virksomheder,475,640,552,541,646,540,557,422,568,615,646,666,503,682,708,2200,m DKK
4,Sektorer i alt,EU,453,463,538,584,593,779,896,960,980,1048,1188,1225,1205,1356,1446,1452,m DKK
5,Sektorer i alt,Finansieringskilder i alt,5291,5897,6811,7691,7929,8514,9086,9244,9736,9940,10685,11091,11005,11893,12497,13159,m DKK
6,Sektorer i alt,Forskningsråd,1243,1358,1687,2056,1886,2170,2138,2205,2177,2199,2065,1975,1748,1689,1750,1941,m DKK
7,Sektorer i alt,"Organisationer, fonde mv.",1128,1088,1391,1647,1764,1932,2221,2304,2596,2731,3128,3793,4077,4613,5197,3987,m DKK
8,Sektorer i alt,Udenlandske virksomheder,136,102,208,255,226,186,256,229,375,485,595,473,542,520,531,649,m DKK


In [20]:
# 4.b Define parameters to display and load in from DST
params = {
    'table': 'FOUOFF09',
    'format': 'CSV',
    'variables': [
        #{'code': 'SEKTOR', 'values': ['1000']},
        {'code': 'VIDOMR', 'values': ['*']},
        #{'code': 'FINANSKILDE', 'values': ['*']},
        {'code': 'Tid', 'values': ['*']}
    ]
}

df_FOU09b = FOU09.get_data(params=params, as_DataFrame=True)

# Displaying results
print("DataFrame with specified parameters:")

display(df_FOU09b)
# display(df.head(10)) 

DataFrame with specified parameters:


,VIDOMR,TID,INDHOLD
0,Naturvidenskab,2007,1709
1,Naturvidenskab,2008,1617
2,Naturvidenskab,2009,1731
3,Naturvidenskab,2010,1979
4,Naturvidenskab,2011,1986
...,...,...,...
91,Humaniora,2018,504
92,Humaniora,2019,506
93,Humaniora,2020,509
94,Humaniora,2021,511


In [21]:
# 5.b Data manipulation and variable construction

# Pivot the DataFrame
df_FOU09b = df_FOU09b.pivot(index=['VIDOMR'], columns='TID', values='INDHOLD')

# Reset index to make TRANSAKT and PRISENHED columns again
df_FOU09b = df_FOU09b.reset_index()

# Add the unit information as a new column
if unit:
    df_FOU09b['Enhed'] = unit

# Display the DataFrame
display(df_FOU09b)

TID,VIDOMR,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,Unit
0,Humaniora,256,297,320,458,381,457,436,464,438,473,491,504,506,509,511,592,m DKK
1,Jordbrugs- og veterinærvidenskab,652,603,678,697,692,610,667,793,838,786,794,742,728,824,893,831,m DKK
2,Naturvidenskab,1709,1617,1731,1979,1986,2158,2235,2307,2454,2384,2669,2951,2880,3199,3162,3438,m DKK
3,Samfundsvidenskab,448,647,703,886,906,985,993,1077,1077,1066,1027,1094,1188,1155,1305,1382,m DKK
4,Sundhedsvidenskab,1577,1941,2269,2437,2651,2920,3266,3135,3327,3542,3971,3865,4237,4506,4776,4898,m DKK
5,Teknisk videnskab,649,792,1110,1234,1313,1384,1489,1468,1602,1689,1733,1935,1466,1700,1850,2018,m DKK


In [22]:
# 6. Download data to excel
output_path = "/Users/ssf/Documents/Analyse/Offentlige udgifter/Test export DST/df_FOU09a.xlsx"
df_FOU09a.to_excel(output_path, index=False)

output_path = "/Users/ssf/Documents/Analyse/Offentlige udgifter/Test export DST/df_FOU09b.xlsx"
df_FOU09b.to_excel(output_path, index=False)

print("DataFrame has been exported to '/Users/ssf/Documents/Analyse/Offentlige udgifter/Test export DST/df_FOU09....xlsx'.")

DataFrame has been exported to '/Users/ssf/Documents/Analyse/Offentlige udgifter/Test export DST/df_FOU09....xlsx'.


**b. RDCE05: Udgifter til FoU efter sektor og videnskabeligt hovedområde**

In [23]:
# 1. Get the table summary from the HELPER CLASS
Udgifter_FOU.tablesummary(language='en')

Table RDCE05: Expenses to R&D by sector, field of research and developement and time
Last update: 2023-12-14T08:00:00


,variable name,# values,First value,First value label,Last value,Last value label,Time variable
0,SEKTOR,5,010,Total source of funding,130,Private non-profit sector,False
1,VIDENHOVED,6,700,Natural sciences,750,Humanities and the arts,False
2,Tid,15,2008,2008,2022,2022,True


In [24]:
# 2. Get the variable levels from the HELPER CLASS - Use this to figure out whch variables we want to use/get from DST.
Udgifter_FOU.variable_levels('SEKTOR', language='en')
Udgifter_FOU.variable_levels('VIDENHOVED', language='en')
#Udgifter_FOU.variable_levels('Tid', language='en')

,id,text
0,700,Natural sciences
1,710,Engineering and technology
2,720,Medical and health sciences
3,730,Agricultural and veterinary sciences
4,740,Social sciences
5,750,Humanities and the arts


In [25]:
# 3. Fetch the unit for the table
unit = Udgifter_FOU.get_table_unit(language='en')
print(f"Unit of measurement: {unit}")

Unit of measurement: m DKK


In [26]:
# 4. Define parameters to display and load in from DST
params = {
    'table': 'RDCE05',
    'format': 'CSV',
    'variables': [
        {'code': 'SEKTOR', 'values': ['110', '120', '130', '010']},
        {'code': 'VIDENHOVED', 'values': ['*']},
        {'code': 'Tid', 'values': ['*']}
    ]
}

df_Udgifter_FOU = Udgifter_FOU.get_data(params=params, as_DataFrame=True)

# Displaying results
print("DataFrame with specified parameters:")

display(df_Udgifter_FOU)
# display(df.head(10)) 


DataFrame with specified parameters:


,SEKTOR,VIDENHOVED,TID,INDHOLD
0,Offentlige myndigheder/institutioner,Naturvidenskab,2008,235
1,Offentlige myndigheder/institutioner,Naturvidenskab,2009,241
2,Offentlige myndigheder/institutioner,Naturvidenskab,2010,223
3,Offentlige myndigheder/institutioner,Naturvidenskab,2011,228
4,Offentlige myndigheder/institutioner,Naturvidenskab,2012,293
...,...,...,...,...
355,I alt sektor,Humaniora,2018,..
356,I alt sektor,Humaniora,2019,1722
357,I alt sektor,Humaniora,2020,..
358,I alt sektor,Humaniora,2021,1820


In [27]:
# 5. Data manipulation and variable construction

# Pivot the DataFrame
df_Udgifter_FOU = df_Udgifter_FOU.pivot(index=['SEKTOR', 'VIDENHOVED'], columns='TID', values='INDHOLD')

# Reset index to make TRANSAKT and PRISENHED columns again
df_Udgifter_FOU = df_Udgifter_FOU.reset_index()

# Add the unit information as a new column
if unit:
    df_Udgifter_FOU['Enhed'] = unit

# Display the DataFrame
display(df_Udgifter_FOU)

TID,SEKTOR,VIDENHOVED,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,Unit
0,Højere læreanstalter og universitetshospitaler,Humaniora,1119,1139,1285,1090,1338,1327,1236,1387,1410,1523,1429,1341,1358,1437,1591,m DKK
1,Højere læreanstalter og universitetshospitaler,Jordbrugs- og veterinærvidenskab,1266,1528,1513,1393,1424,1282,1366,1500,1203,1237,1176,1239,1108,1330,1447,m DKK
2,Højere læreanstalter og universitetshospitaler,Naturvidenskab,3115,3199,3515,3552,3637,3808,3696,4402,4444,4798,5136,5633,5095,5508,6031,m DKK
3,Højere læreanstalter og universitetshospitaler,Samfundsvidenskab,1837,2065,2373,2555,2421,2864,3066,3147,3071,2630,3100,3142,3221,3646,3895,m DKK
4,Højere læreanstalter og universitetshospitaler,Sundhedsvidenskab,4169,4399,4860,5702,6176,7141,7078,6788,7151,7095,7375,7574,8683,9895,11053,m DKK
5,Højere læreanstalter og universitetshospitaler,Teknisk videnskab,2010,2257,2469,2501,2879,3002,2915,3892,3821,4072,4257,4192,4404,4665,4781,m DKK
6,I alt sektor,Humaniora,..,..,..,..,..,..,..,..,..,2039,..,1722,..,1820,..,m DKK
7,I alt sektor,Jordbrugs- og veterinærvidenskab,..,..,..,..,..,..,..,..,..,1747,..,1773,..,1668,..,m DKK
8,I alt sektor,Naturvidenskab,..,..,..,..,..,..,..,..,..,11033,..,13724,..,12669,..,m DKK
9,I alt sektor,Samfundsvidenskab,..,..,..,..,..,..,..,..,..,3397,..,3835,..,4260,..,m DKK


In [28]:
# 6. Download data to excel
output_path = "/Users/ssf/Documents/Analyse/Offentlige udgifter/Test export DST/df_Udgifter_FOU.xlsx"
df_Udgifter_FOU.to_excel(output_path, index=False)

print("DataFrame has been exported to '/Users/ssf/Documents/Analyse/Offentlige udgifter/Test export DST/df_Udgifter_FOU.xlsx'.")

DataFrame has been exported to '/Users/ssf/Documents/Analyse/Offentlige udgifter/Test export DST/df_Udgifter_FOU.xlsx'.


**c. COMBINING DATA IN THE EXCEL FILE *"Forskningsudgifter samlet (DST).xlsx"*.**

In [29]:
# Path to the output Excel file
output_path = "/Users/ssf/Documents/Analyse/Offentlige udgifter/Test export DST/Forskningsudgifter samlet (DST).xlsx"

# Using ExcelWriter to write multiple DataFrames to different sheets
with pd.ExcelWriter(output_path) as writer:
    df_MAKRO.to_excel(writer, sheet_name='NAN1', index=False)
    df_FOU07.to_excel(writer, sheet_name='FOUOFF07', index=False)
    df_FOU09a.to_excel(writer, sheet_name='FOUOFF09_Sektor', index=False)
    df_FOU09b.to_excel(writer, sheet_name='FOUOFF09_Fag', index=False)
    df_Udgifter_FOU.to_excel(writer, sheet_name='RDCE05', index=False)

print(f"DataFrames have been exported to '{output_path}' with multiple sheets.")

DataFrames have been exported to '/Users/ssf/Documents/Analyse/Offentlige udgifter/Test export DST/Forskningsudgifter samlet (DST).xlsx' with multiple sheets.


# RAW FUI-TAL DST

Kan nok ikke importeres som API, da de ikke er på statistikbanken.